In [1]:
#Import Dependencies
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#import datasets
gdp = pd.read_csv("./resources/gdp.csv")
protest = pd.read_csv("./resources/world_protest.csv")

### Clean GDP dataframe
* convert from wide format to long format using pandas
* Rename column headers and reset index

*Note: For country/years with NaN, we decided to leave in NaN rather than convert to 0. NaN represents a null value for which data was not collected that year by the world bank. A value of 0 would represent a 0% GDP and could complicate downstream analysis*

In [3]:
#Add an id column
gdp["id"] = gdp.index

In [4]:
#Use pandas method to convert dataframe from wide to long based off of year
gdp = pd.wide_to_long(gdp, ["X"], i = "id", j="year")

In [10]:
#Explain why we are leaving in the NaN
gdp_df = gdp.rename(columns= {"X":"gdp", "Country Code": "country_code", "Country Name": "country_name"}).reset_index()

In [27]:
gdp_df = gdp_df[["year", "country_code", "country_name", "gdp"]]

year               59
country_code      264
country_name      264
gdp             11654
dtype: int64

## Clean Mass Protest dataframe
* Select columns of interest
* Group data by country and year to get the total number of protest each year 
* Reset index

In [20]:
protest = protest[["id", "country", "year", "protest"]]
protest_df = protest.groupby(["country", "year"]).sum()["protest"].reset_index()
protest_df

,country,year,protest
0,Afghanistan,1990,0
1,Afghanistan,1991,1
2,Afghanistan,1992,0
3,Afghanistan,1993,0
4,Afghanistan,1994,0
...,...,...,...
4712,Zimbabwe,2015,3
4713,Zimbabwe,2016,17
4714,Zimbabwe,2017,5
4715,Zimbabwe,2018,5


### Create merged database to get year/gdp/sum protest in single graph

### Connect to PostgreSQL database on pgAdmin

In [28]:
rds_connection_string = "postgres:lms9878@localhost:5434/etl_mass_gdp"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [29]:
engine.table_names()

['gdp', 'mass_protest']

### Load CSV converted DataFrame into database using pandas

In [30]:
gdp_df.to_sql(name='gdp', con=engine, if_exists='append', index=False)
protest_df.to_sql(name='mass_protest', con=engine, if_exists='append', index=False)